<a href="https://colab.research.google.com/github/mariumabid/AI-challenge-/blob/master/1_Data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
# The data URI
csv_file_uri = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
csv_file_uri = "adult.data.csv"

### Data overview

```
>50K, <=50K.

age: continuous.
workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
fnlwgt: continuous.
education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
education-num: continuous.
marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
sex: Female, Male.
capital-gain: continuous.
capital-loss: continuous.
hours-per-week: continuous.
native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
```

Loading the data

In [0]:
column_names = [
    "age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
    "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
    "hours-per-week", "native-country", "target"
]


data_original = pd.read_csv(csv_file_uri, names=column_names, index_col=False)
# Make a copy so that we always have the original data to refer to
data = data_original.copy(deep=True)
# Drop the US weights (don't have any value)
data.drop(["fnlwgt"], axis=1, inplace=True)

# Show the head rows of the table at this stage.
data.head(3)

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [0]:
# How big is the dataset?
data.shape

The variable we want to predict for classification is the **target**. The problem now is that it is text and we can not work with text in most models.

Here is one way to change a column in a pandas dataframe (using the `apply` method)

In [0]:
# Create a function that changes the text to a simple binary value
def convert_target_variable(text):
    if text == " <=50K":
        return 0
    else:
        return 1

data["target_encoded"] = data.target.apply(convert_target_variable)

# Deletes the original column in this dataframe.
data.drop(["target"], axis=1, inplace=True)

data.head(3)

In [0]:
# To check how many people make less than 50k and how many make more
data.target_encoded.value_counts()

### Data Preprocessing

At this point we have a big problem with our data. Algorithms (most of them) can only handle data (as they rely on mathematics). For this reason we need to get rid of any column that is a text variable and change it to a numerical one.

You can use one of many classes from the machine learning toolkit **scikit-learn**

In [0]:
from sklearn import preprocessing

In [0]:
encoder = preprocessing.LabelEncoder()

data["race_encoded"] = encoder.fit_transform(data.race.values)
data.drop(["race"], axis=1, inplace=True)

# View your new column.
data.head(3)

**Discuss what a Label Encoder does, make sure you understand how it works**

We have a whole bunch of these columns (run the following block to see which), so lets encode them all.

In [0]:
data.dtypes

In [0]:
encoded_columns = []
for c in data.columns:
    if data[c].dtype == "object":
        if "{}_encoded".format(c) not in data.columns:
            encoder = preprocessing.LabelEncoder()
            data["{}_encoded".format(c)] = encoder.fit_transform(data[c].values)
            encoded_columns.append(c)
            encoder = None
        else:
            print("{}_encoded already exists".format(c))

print("Dropping the encoded columns {}".format(encoded_columns))
data.drop(encoded_columns, axis=1, inplace=True)

In [0]:
# Check out the new numerical data table.
data.head()

In [0]:
# All available column names
data.columns

### Challenge

In [0]:
# Don't use the LabelEncoder but use one-hot-encoding instead.
# For this you will need to use the pandas function pd.get_dummies
# to encode and either a dataframe join or merge to merge the dataframes
# inside the loop.